# Домашнее задание

Работаем с датасетом про дорожные знаки.

https://www.kaggle.com/datasets/valentynsichkar/traffic-signs-preprocessed/data

В первых двух задачах берем только файл train.pickle и можно подгрузить labels_names.csv

Данные можно прочитать через `pd.read_pickle`

## Easy

Скачать датасет, обучить логистическую регрессию, измерить качество.

Текстом кратко обосновать какую метрику выбираем и почему такую.

Какой вид дорожных знаков модель путает чаще всего?

## Medium

Обучить и сравнить между собой:

- логистическая регрессия
- kNN
- НБК
- SVM (попробовать 3 разных ядра)

Какая модель отработала лучше, а какая хуже всех?

Проинтерпретируйте почему худшая модель именно эта.

## Hard

Вы - ML-инженер в команде беспилотных автомобилей! Ваша следующая цель - сделать алгоритм, который будет распознавать дорожные знаки. Другая команда уже позаботилась о подготовке данных - вам приходят обрезанные картинки 32х32. Но есть проблема - разметка стоит денег, у проекта есть бюджетные ограничения, но вам готовы выделить тот бюджет, который вы сможете обосновать.

Определите сколько и каких семплов данных необходимо для того, чтобы обучить достаточно хороший алгоритм.

**Подсказка:** При анализе изображений обычно делают специальную предобработку - картинки крутят-вертят, по-разному меняют цвета. Рядом с данными лежит питоновский скрипт, может быть он поможет уменьшить необходимое количество семплов?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

C:\Users\szabu\AppData\Local\Temp\ipykernel_9560\1906245463.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Easy

In [2]:
train_dict = pd.read_pickle("../data/train.pickle")

df_train = pd.DataFrame((train_dict['features']).reshape(train_dict['features'].shape[0], -1))

df_train['y'] = train_dict['labels']

df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,y
0,28,25,24,27,24,23,27,24,22,27,...,27,24,22,27,23,21,26,22,20,41
1,28,24,24,26,23,23,27,24,24,28,...,27,23,22,27,24,23,29,26,25,41
2,29,25,25,29,26,26,30,27,27,30,...,29,25,23,28,24,22,30,26,24,41
3,31,28,26,29,26,24,30,27,26,29,...,28,23,22,27,23,22,27,23,21,41
4,33,27,25,31,25,24,32,26,25,31,...,25,22,20,26,22,19,27,22,19,41


In [3]:
test_dict = pd.read_pickle("../data/test.pickle")

df_test = pd.DataFrame((test_dict['features']).reshape(test_dict['features'].shape[0], -1))

df_test['y'] = test_dict['labels']

df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,y
0,116,139,174,116,137,171,118,138,172,117,...,115,138,167,117,138,167,114,140,170,16
1,59,70,61,87,80,63,92,81,63,96,...,62,59,62,73,67,70,75,70,70,1
2,52,40,38,51,39,37,50,38,37,50,...,41,40,38,44,43,43,47,46,44,38
3,132,125,87,100,92,60,82,73,34,96,...,235,239,196,255,251,217,255,240,180,33
4,37,34,30,42,38,32,46,40,36,44,...,46,41,38,48,43,40,49,45,42,11


In [4]:
model = LogisticRegression(max_iter=100)
model.fit(df_train.drop(columns=['y']), df_train['y'])

C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [5]:
preds = model.predict(df_test.drop(columns=['y']))

# Хотим оценить, как в целом справляется модель, и посчитаем precision/recall на случай дизбаланса классов
# (На следующей ячейке еще confusion matrix считается)
print(f"Accuracy: {accuracy_score(preds, df_test['y'])}")
print(f"Recall: {recall_score(preds, df_test['y'], average='macro')}")
print(f"Precision: {precision_score(preds, df_test['y'], average='macro')}")

Accuracy: 0.8169437846397466
Recall: 0.7928267421239333
Precision: 0.7399774952886683


In [6]:
conf_matrix = confusion_matrix(preds, df_test['y'])

errors = []
for i in range(43):
    for j in range(43):
        if i != j:
            # Смотрим, что сколько раз модель запуталась, и что конкретно перепутала
            errors.append((conf_matrix[i][j], i, j))
        
# Возьмем топ-10 результатов
most_confusion = sorted(errors, reverse=True)[:10]
most_confusion

[(76, 1, 2),
 (70, 1, 4),
 (63, 5, 3),
 (56, 2, 1),
 (55, 8, 7),
 (52, 5, 7),
 (49, 2, 5),
 (38, 30, 23),
 (30, 12, 21),
 (29, 4, 17)]

In [7]:
label_names = pd.read_csv("../data/label_names.csv")

most_confusion_names = [(label_names['SignName'].iloc[e[1]], label_names['SignName'].iloc[e[2]]) for e in most_confusion]
most_confusion_names  # У модели явно беды с распознаванием цифр на ограничительных знаках. :)

[('Speed limit (30km/h)', 'Speed limit (50km/h)'),
 ('Speed limit (30km/h)', 'Speed limit (70km/h)'),
 ('Speed limit (80km/h)', 'Speed limit (60km/h)'),
 ('Speed limit (50km/h)', 'Speed limit (30km/h)'),
 ('Speed limit (120km/h)', 'Speed limit (100km/h)'),
 ('Speed limit (80km/h)', 'Speed limit (100km/h)'),
 ('Speed limit (50km/h)', 'Speed limit (80km/h)'),
 ('Beware of ice/snow', 'Slippery road'),
 ('Priority road', 'Double curve'),
 ('Speed limit (70km/h)', 'No entry')]

# Medium

In [8]:
X_train, y_train, X_test, y_test = df_train.drop(columns=['y']), df_train['y'], df_test.drop(columns=['y']), df_test['y']

In [9]:
models = {
    'Logistic Regression': LogisticRegression(),
    'kNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'SVM_linear': SVC(kernel='linear', max_iter=100),
    'SVM_poly': SVC(kernel='poly', max_iter=100),
    'SVM_rbf': SVC(kernel='rbf', max_iter=100)
}

In [10]:
for name, model in models.items():
    print(f"Training {name}:")
    model.fit(X_train, y_train)
    print("Training finished!")

Training Logistic Regression:


C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Training finished!
Training kNN:
Training finished!
Training Naive Bayes:
Training finished!
Training SVM_linear:


C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Training finished!
Training SVM_poly:


C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Training finished!
Training SVM_rbf:
Training finished!


C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [11]:
# Оценка производительности
results = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}

C:\Users\szabu\anaconda3\envs\classical-ml\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Вывод результатов
for name, metrics in results.items():
    print(f'{name}: \n{metrics}')

Logistic Regression: 
{'Accuracy': 0.8169437846397466, 'Precision': 0.820744687791157, 'Recall': 0.8169437846397466, 'F1': 0.8145731105636637}
kNN: 
{'Accuracy': 0.3847189231987332, 'Precision': 0.423392251068593, 'Recall': 0.3847189231987332, 'F1': 0.39095958617170934}
Naive Bayes: 
{'Accuracy': 0.11971496437054632, 'Precision': 0.3369179056192524, 'Recall': 0.11971496437054632, 'F1': 0.13146324342826704}
SVM_linear: 
{'Accuracy': 0.6564528899445764, 'Precision': 0.6798514952516299, 'Recall': 0.6564528899445764, 'F1': 0.653418914847792}
SVM_poly: 
{'Accuracy': 0.1564528899445764, 'Precision': 0.4127479846556274, 'Recall': 0.1564528899445764, 'F1': 0.16062015486350928}
SVM_rbf: 
{'Accuracy': 0.35930324623911325, 'Precision': 0.5213108497778621, 'Recall': 0.35930324623911325, 'F1': 0.3626512419767831}


## Выводы после обучения
Лучше всего показали себя логистическая регрессия и SVM с линейным ядром, что может означать наличие линейных зависимостей между пикселями и целевой переменной.

Предположения, почему другие модели справились не так хорошо:

**KNN**: kNN чувствителен к размерности данных, которая в нашем случае составляет 32 * 32 * 3, что немало. 

**NBC**: эта модель хорошо работает, если фичи независимы, что, вероятно, не так.

**SVM (другие ядра)**: сложно сказать, но возможно, просто дефолтные параметры плохо справляются. Может, пытаться находить более сложные зависимости на этих данных -- плохая идея...

P.S. При max_iter = -1 результаты для logreg и линейной SVM почти совпадают (метрики около 0.8), но при перезапуске ячеек я ограничил количество итераций до 100, чтобы не слишком долго ждать.